***RNN with Multi-class classification***

In [2]:
# RNN 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Load data from CSV file
df = pd.read_csv('Bug_Reports_Final_data.csv')

# Select a subset of the data for testing purposes
df = df.sample(n=1000, random_state=42)

# Preprocess the text data
max_words = 10000
maxlen = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['stem'])
sequences = tokenizer.texts_to_sequences(df['stem'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)

# Convert label to categorical
labels = to_categorical(np.asarray(df['label'].astype('category').cat.codes))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)

# Build the RNN model
embedding_size = 100
model = Sequential()
model.add(Embedding(max_words, embedding_size, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Train the model
batch_size = 50 
epochs = 10
model.fit(X_train, y_train, validation_split=0.2, batch_size=batch_size, epochs=epochs)

# Evaluate the model on the test set
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
# Generate predictions and print classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
print(classification_report(y_test_classes, y_pred_classes))

Found 5173 unique tokens.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 100)         1000000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 1,117,764
Trainable params: 1,117,764
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
13/13 [==============================] - 51s 4s/step - loss: 1.2711 - accuracy: 0.3734 - val_loss: 1.0777 - val_accuracy: 0.4938
Epoch 2/10
13/13 [==============================] - 49s 4s/step - loss: 0.9643 - accuracy: 0.5859 - val_loss: 0.7873 - val_accuracy: 0.62